Il est intéressant de constater qu'en apprentissage profond, l'évolution des architecture et des techniques est très rapide, mais que toutefois certains aspects peuvent rester des années présent dans l'ensemble des codes, sans savoir vraiment s'ils sont totalement nécessaire.

Un de ces éléments et la normalisation qui est utilisé en entrée de la plupart des réseaux convolutionnels qui servent à traiter des images visuelles. Ceci est d'autant plus vrai que nous utilisons souvent dans la communauté des réseaux qui sont pré entraînés pour des images normalisé à des valeurs données.
Ceci est d'autant plus vrai que nous utilisons souvent dans la communauté des réseaux qui sont pré entraînés pour des images normalisé à des valeurs données.

Je m'intéresse ici de savoir si ces réseaux restent efficaces quand on change la normalisation en entrée. En effet, comme les poids de la première couche convolutionnelle sont appris, et que notamment grâce au biais dans les convolutions ainsi que les noyaux de convolution puissent être multiplié par des valeurs arbitraires, une normalisation vers une certaine valeur n'a a priori aucun sens, et simplement le fait de normaliser en a un. Je me propose ici de montrer quantitativement que c'est le cas, d'abord pour un réseau *historique* LeNet appliqué au challenge MNIST, et ensuite à ResNet appliqué à ImageNet.


<!-- TEASER_END -->

Let's first initialize the notebook:

In [ ]:
from __future__ import division, print_function
import numpy as np
np.set_printoptions(precision=6, suppress=True)
import os
%matplotlib inline
#%config InlineBackend.figure_format='retina'
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
phi = (np.sqrt(5)+1)/2
fig_width = 10
figsize = (fig_width, fig_width/phi)
from IPython.display import display, HTML
def show_video(filename): 
    return HTML(data='<video src="{}" loop autoplay width="600" height="600"></video>'.format(filename))
%load_ext autoreload
%autoreload 2

# MNIST

Explorons d'abord, le réseau *leNet*, dont l'objectif est de catégoriser, des images de lettres dactylographiées, une des premières grandes réussites des réseaux de neurones Multicouche. Nous allons pour celui-là utiliser l'implémentation classique, tel qu'elle est proposé dans la série des exemples de la librairie PyTorch. 

La cellule ci-dessous permet de tout faire : d'abord charger les librairies, ensuite définir le réseau de neurones, et enfin définir la procédure d'apprentissage et de test qui sont appliquées à la base de données. On obtient en sortie une valeur d'Accura qui correspond au pourcentage de lettres qui sont correctement classifier.
.

Dans cette cellule, on va isoler les deux paramètres qui vont permettre de régler la moyenne ainsi que la déviation standard qui sont appliquée à la fonction de normalisation, et que nous allons manipuler au cours de ce book


In [2]:
# adapted from https://raw.githubusercontent.com/pytorch/examples/refs/heads/main/mnist/main.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

epochs = 15

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if verbose and batch_idx % log_interval  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, verbose=True):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    if verbose:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / len(test_loader.dataset)


def main(mean, std, epochs=15, log_interval=100, verbose=True):
    # Training settings
    use_cuda = torch.cuda.is_available()
    use_mps = torch.backends.mps.is_available()

    torch.manual_seed(1998) # FOOTIX rules !

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': 64}
    test_kwargs = {'batch_size': 1000}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((mean,), (std,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)

    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=1.0)

    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
    for epoch in range(1, epochs):
        train(model, device, train_loader, optimizer, epoch, log_interval=log_interval, verbose=verbose)
        scheduler.step()

    accuracy = test(model, device, test_loader, verbose=verbose)
    return accuracy



Maintenant que nous avons défini l'ensemble du protocole, nous avons pouvoir le tester tout d'abord dans sa formule la plus classique, telle qu'elle est livrée dans la librairie :

In [ ]:
accuracy = main(mean=0.1307, std=0.3081)
print(f'{accuracy=:.4f}')


Un avantage de notre code et maintenant de pouvoir manipuler ces deux valeurs, afin de voir si en partant du même code on obtient une valeur d'accuracy qui est différente.


Le résultat semble similaire, mais ce n'est pas assez pour démontrer que la moyenne est la déviation standard n'ont pas d'effets sur l'apprentissage. Je vais maintenant utiliser les grands moyens en utilisant une librairie qui permet de tester de multiples valeurs et ainsi d'optimiser les paramètres. Si la moyenne est la déviation sont si important. Alors nous allons converger vers un ensemble de valeur fixe, alors que si c'est moins important, les valeurs pourront être assez dispersées. Cela nous permettra en particulier de choisir une valeur arbitraire comme par exemple une moyenne de zéro et une déviation standard de zéro, ce qu'on appelle une normalisation *normale*.


In [5]:
accuracy = main(mean=0., std=1., epochs=epochs, verbose=False)
print(f'{accuracy=:.4f}')


accuracy=0.9909


In [6]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
path_save_optuna =  'optuna-MNIST.sqlite3'

In [7]:
%rm {path_save_optuna}

In [ ]:
def objective(trial):
    mean = trial.suggest_float('mean', 0.001, 10, log=True)
    std = trial.suggest_float('std', 0.001, 10, log=True)
    accuracy = main(mean=mean, std=std, epochs=epochs, verbose=False)
    return accuracy


if not(os.path.isfile(path_save_optuna)):
    study = optuna.create_study(direction='maximize', load_if_exists=True, 
                                storage=f"sqlite:///{path_save_optuna}", study_name='MNIST')
    study.optimize(objective, n_trials=200, n_jobs=-1, show_progress_bar=True)
    print(50*'-.')
    print("Best params: ", study.best_params)
    print("Best value: ", study.best_value)
    print("Best Trial: ", study.best_trial)
    print(50*'=')
    # print("Trials: ", study.trials)


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
# https://optuna.readthedocs.io/en/stable/reference/visualization/matplotlib/generated/optuna.visualization.matplotlib.contour.html
from optuna.visualization.matplotlib import plot_contour

fig = plot_contour(study, params=["mean", "std"], target_name="Accuracy")
fig.set_size_inches(figsize)
fig

## some book keeping for the notebook

In [ ]:
%load_ext watermark
%watermark -i -h -m -v -p numpy,matplotlib,torch  -r -g -b